# Solving Sudoku with Algebraic Geometry and Computer Algebra : A C Programming Approach. 

Source [tetsuo.ai](https://x.com/7etsuo/status/1813482989139161344)

## 1. Algebraic Formulation

$$
K \in \{ \mathbb{Q}, \text{finite field} \} 
$$
### Definition 1.1:
$$
(x_1, \dots , x_{81}) \in \text{Sudoku grid variables}
$$
### Definition 1.2:
$$
\forall i \in \{1, \dots, 81\}, F_i \in K[x_i], F_i(x_i) = \prod_{k=1}^{9} (x_i - k)
$$

```c
int F(int x)
{
    int result = 1;
    for (int k = 1; k <= 9; k++)
        result *= (x - k);
    return result;
}
```
### Lemma 1.3:
$$
F_i(a_i) = 0 \iff a_i \in \{1, \dots, 9\}
$$
### Definition 1.4:
$$
i \neq j, \quad G_{ij}(x_i, x_j) = \frac{F_i(x_i) - F_j(x_j)}{x_i - x_j} \in K[x_i, x_j]
$$

```c
int G(int xi, int xj)
{
    if (xi == xj)
        return 0; // Avoid division by zero
    return (F(xi) - F(xj)) / (xi - xj);
}
```
### Lemma 1.5:
$$
G_{ij}(a_i, a_j) = 0 \iff a_i = a_j \text{or } (a_i, a_j \in \{1, \dots, 9\} \text{ and } a_i \neq a_j)
$$
### Definition 1.6:
$$
E = \{ (i, j) \mid 1 \leq i < j \leq 81, \text{same row, column, or } 3 \times 3 \text{ block} \}
$$

```c
typedef struct
{
    int *data;
    int size;
    int capacity;
} IntPairList;

IntPairList createE(void)
{
    IntPairList E;
    initIntPairList(&E);
    int i, j, k, l, a, b;

    for (j = 1; j <= 9; j++)
    {
        for (k = 1; k <= 9; k++)
        {
            i = (j - 1) * 9 + k;
            a = j % 3;
            if (a == 0)
                a = 3;
            b = k % 3;
            if (b == 0)
                b = 3;
            /* Same row */
            for (l = k + 1; l <= 9; l++)
                addPair(&E, i, i + l - k);
            /* Same column */
            for (l = j + 1; l <= 9; l++)
                addPair(&E, i, (l - 1) * 9 + k);
            /* Same 3x3 block */
            if (a != 3)
                if (b != 3)
                    addPair(&E, i, j * 9 + k + 1);
            if (b != 2 && b != 3)
                addPair(&E, i, j * 9 + k + 2);
            if (a == 1 || a == 2)
                if (b == 1)
                {
                    addPair(&E, i, (j + 1) * 9 + k);
                    addPair(&E, i, (j + 1) * 9 + k + 1);
                    addPair(&E, i, (j + 1) * 9 + k + 2);
                }
                else if (b == 2)
                {
                    addPair(&E, i, (j + 1) * 9 + k - 1);
                    addPair(&E, i, (j + 1) * 9 + k);
                    addPair(&E, i, (j + 1) * 9 + k + 1);
                }
                else
                { // b == 3
                    addPair(&E, i, (j + 1) * 9 + k - 2);
                    addPair(&E, i, (j + 1) * 9 + k - 1);
                    addPair(&E, i, (j + 1) * 9 + k);
                }
            if (a == 1)
            {
                if (b == 1)
                {
                    addPair(&E, i, (j + 2) * 9 + k);
                    addPair(&E, i, (j + 2) * 9 + k + 1);
                    addPair(&E, i, (j + 2) * 9 + k + 2);
                }
                else if (b == 2)
                {
                    addPair(&E, i, (j + 2) * 9 + k - 1);
                    addPair(&E, i, (j + 2) * 9 + k);
                    addPair(&E, i, (j + 2) * 9 + k + 1);
                }
                else
                { /* b == 3 */
                    addPair(&E, i, (j + 2) * 9 + k - 2);
                    addPair(&E, i, (j + 2) * 9 + k - 1);
                    addPair(&E, i, (j + 2) * 9 + k);
                }
            }
        }
    }
    return E;
}
```

### Definition 1.7:
$$
I \subset K[x_1, \dots, x_{81}] \text{ideal generated by } \{ F_i \mid i = 1, \dots, 81 \} \cup \{ G_{ij} \mid (i, j) \in E \}
$$


## 2. Key Propositions

### Proposition 2.1:

$$
V(I) \text{ vanishing locus of } I \text{ in } A_{81}(K), a = (a_1, \dots, a_{81}) \in A_{81}(K) \\
$$

Then,
$$
a \in V(I) \iff \left( a_i \in \{1, \dots, 9\} \, \forall i = 1, \dots, 81 \right) \text{and } \left( a_i \neq a_j \, \forall (i, j) \in E \right)
$$

#### Proof:

$$
\text{If conditions } (1) \text{ and } (2) \text{ are fulfilled, then } F_i \text{ and } G_{ij} \text{ vanish at } a, \text{so } a \in V(I). \\
$$

Conversely,
$$
a \in V(I) \implies F_i(a_i) = 0 \implies a_i \in \{1, \dots, 9\} \, \forall i
$$

$$
\text{To show } a_i \neq a_j \text{ for } (i, j) \in E, \text{suppose } a_i = a_j = b \text{ for some } (i, j)
$$

$$
\text{Substituting } b \text{ for } x_j \text{ in } F_i(x_i) = (x_i - x_j) G_{ij}(x_i, x_j) + F_j(x_j)
$$

gives:
$$
F_i(x_i) = (x_i - b) G_{ij}(x_i, b)
$$

Since,  
$$
G_{ij}(b, b) = 0
$$

by assumption, this would imply that

$$
b \text{ is a zero of } F_i
$$

of order at least two which is impossible.  QED

### Definition 2.2:

A Sudoku is well-posed if it has a unique solution.

### Proposition 2.3:

$$
S = \{ a_i \} \, i \in L, \quad L \subset \{1, \dots, 81\}
$$

$$
I_S = I + \langle x_i - a_i \mid i \in L \rangle
$$

$$
\text{Gröbner basis of } I_S : (x_1 - a_1, \dots, x_{81} - a_{81}) \quad (a_1, \dots, a_{81}) \text{ is the solution}
$$

#### Proof:

By Proposition 1.1 and the Nullstellensatz,

$$
S \text{ has a unique solution } (a_1, \dots, a_{81}) \implies I_S = \langle x_1 - a_1, \dots, x_{81} - a_{81} \rangle
$$

$$
I_S \supseteq (x_i - a_i)^m \quad \forall i \text{ for some } m
$$

$$
I_S \supseteq F_i(x_i) = \prod_{k=1}^{9} (x_i - k) \implies I_S \cap K[x_i] = \langle x_i - a_i \rangle \quad \forall i
$$

$$
\therefore I_S = \langle x_1 - a_1, \dots, x_{81} - a_{81} \rangle
$$

, and the shape of the reduced Gröbner basis is as claimed.

## 3. Computational Aspects

The algebraic geometry approach is not the most computationally efficient method for solving Sudoku puzzles. Graph-theoretic methods, which treat Sudoku as a graph coloring problem with nine colors, are much faster. \
For those interested in implementing this method, computer algebra systems like [SINGULAR](https://www.singular.uni-kl.de/) can be used to compute Gröbner bases and solve the resulting systems of polynomial equations.\
For demonstration purposes, here is a Sudoku solver implemented in C. This solver incorporates some of the algebraic concepts discussed earlier and utilizes backtracking for finding solutions:

In [16]:
%%file test_sudoku.c
/********************************************************************
*  ███████████           █████                                      *
* ░█░░░███░░░█          ░░███                                       *
* ░   ░███  ░   ██████  ███████    █████  █████ ████  ██████        *
*     ░███     ███░░███░░░███░    ███░░  ░░███ ░███  ███░░███       *
*     ░███    ░███████   ░███    ░░█████  ░███ ░███ ░███ ░███       *
*     ░███    ░███░░░    ░███ ███ ░░░░███ ░███ ░███ ░███ ░███       *
*     █████   ░░██████   ░░█████  ██████  ░░████████░░██████        *
*    ░░░░░     ░░░░░░     ░░░░░  ░░░░░░    ░░░░░░░░  ░░░░░░         *
*                                                                   *
********************************************************************/

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>

#ifndef N
#define N 9
#endif
#ifndef EMPTY
#define EMPTY 0
#endif

typedef struct
{
    int *data;
    int size;
    int capacity;
} IntPairList;

void InitIntPairList(IntPairList *list);
void AddPair(IntPairList *list, int first, int second);
IntPairList CreateE(void);
void PrintBoard(int board[N][N]);
bool isValid(int board[N][N], int row, int col, int num);
bool FindEmptyCell(int board[N][N], int *row, int *col);
bool SolveSudoku(int board[N][N]);

int main(int argc, char *argv[])
{
    int board[N][N] = {
        {1, 0, 0, 0, 7, 3, 8, 9, 0},
        {0, 0, 0, 0, 0, 2, 0, 0, 0},
        {0, 6, 0, 0, 0, 0, 0, 1, 0},
        {0, 0, 0, 9, 0, 0, 0, 0, 0},
        {0, 0, 4, 0, 0, 0, 0, 0, 5},
        {6, 0, 0, 0, 8, 1, 3, 0, 0},
        {0, 5, 0, 0, 9, 7, 0, 0, 3},
        {7, 0, 0, 2, 0, 0, 0, 0, 0},
        {0, 0, 0, 6, 0, 0, 9, 0, 0}
    };

    printf("Initial Sudoku puzzle:\n");
    PrintBoard(board);

    IntPairList E = CreateE();
    printf("\nSet E created with %d pairs.\n", E.size);

    if (SolveSudoku(board))
    {
        printf("\nSolved Sudoku puzzle:\n");
        PrintBoard(board);
    }
    else
        printf("\nNo solution exists.\n");

    free(E.data);
    return 0;
}

void InitIntPairList(IntPairList *list)
{
    list->size = 0;
    list->capacity = 10;
    list->data = malloc(list->capacity * 2 * sizeof(int));
}

void AddPair(IntPairList *list, int first, int second)
{
    if (list->size >= list->capacity)
    {
        list->capacity *= 2;
        list->data = realloc(list->data, list->capacity * 2 * sizeof(int));
    }
    list->data[list->size * 2] = first;
    list->data[list->size * 2 + 1] = second;
    list->size++;
}

IntPairList CreateE(void)
{
    IntPairList E;
    InitIntPairList(&E);
    int i, j, k, l, a, b;

    for (j = 1; j <= 9; j++)
    {
        for (k = 1; k <= 9; k++)
        {
            i = (j - 1) * 9 + k;
            a = j % 3;
            if (a == 0)
                a = 3;
            b = k % 3;
            if (b == 0)
                b = 3;
            /* Same row */
            for (l = k + 1; l <= 9; l++)
                AddPair(&E, i, i + l - k);
            /* Same column */
            for (l = j + 1; l <= 9; l++)
                AddPair(&E, i, (l - 1) * 9 + k);
            /* Same 3x3 block */
            if (a != 3)
            {
                if (b != 3)
                    AddPair(&E, i, j * 9 + k + 1);
                if (b != 2 && b != 3)
                    AddPair(&E, i, j * 9 + k + 2);
            }
            if (a == 1 || a == 2)
            {
                if (b == 1)
                {
                    AddPair(&E, i, (j + 1) * 9 + k);
                    AddPair(&E, i, (j + 1) * 9 + k + 1);
                    AddPair(&E, i, (j + 1) * 9 + k + 2);
                }
                else if (b == 2)
                {
                    AddPair(&E, i, (j + 1) * 9 + k - 1);
                    AddPair(&E, i, (j + 1) * 9 + k);
                    AddPair(&E, i, (j + 1) * 9 + k + 1);
                }
                else
                { /* b == 3 */
                    AddPair(&E, i, (j + 1) * 9 + k - 2);
                    AddPair(&E, i, (j + 1) * 9 + k - 1);
                    AddPair(&E, i, (j + 1) * 9 + k);
                }
            }
            if (a == 1)
            {
                if (b == 1)
                {
                    AddPair(&E, i, (j + 2) * 9 + k);
                    AddPair(&E, i, (j + 2) * 9 + k + 1);
                    AddPair(&E, i, (j + 2) * 9 + k + 2);
                }
                else if (b == 2)
                {
                    AddPair(&E, i, (j + 2) * 9 + k - 1);
                    AddPair(&E, i, (j + 2) * 9 + k);
                    AddPair(&E, i, (j + 2) * 9 + k + 1);
                }
                else
                { /* b == 3 */
                    AddPair(&E, i, (j + 2) * 9 + k - 2);
                    AddPair(&E, i, (j + 2) * 9 + k - 1);
                    AddPair(&E, i, (j + 2) * 9 + k);
                }
            }
        }
    }
    return E;
}

void PrintBoard(int board[N][N])
{
    printf("-------------------------\n");
    for (int i = 0; i < N; i++)
    {
        printf("| ");
        for (int j = 0; j < N; j++)
        {
            if (board[i][j] == EMPTY)
            {
                printf(". ");
            }
            else
            {
                printf("%d ", board[i][j]);
            }
            if ((j + 1) % 3 == 0)
            {
                printf("| ");
            }
        }
        printf("\n");
        if ((i + 1) % 3 == 0)
        {
            printf("-------------------------\n");
        }
    }
}

bool isValid(int board[N][N], int row, int col, int num)
{
    for (int x = 0; x < N; x++)
        if (board[row][x] == num)
            return false;

    for (int x = 0; x < N; x++)
        if (board[x][col] == num)
            return false;

    int startRow = row - row % 3, startCol = col - col % 3;
    for (int i = 0; i < 3; i++)
        for (int j = 0; j < 3; j++)
            if (board[i + startRow][j + startCol] == num)
                return false;

    return true;
}

bool FindEmptyCell(int board[N][N], int *row, int *col)
{
    for (*row = 0; *row < N; (*row)++)
        for (*col = 0; *col < N; (*col)++)
            if (board[*row][*col] == EMPTY)
                return true;
    return false;
}

bool SolveSudoku(int board[N][N])
{
    int row, col;

    if (!FindEmptyCell(board, &row, &col))
        return true;

    for (int num = 1; num <= 9; num++)
    {
        if (isValid(board, row, col, num))
        {
            board[row][col] = num;

            if (SolveSudoku(board))
                return true;

            board[row][col] = EMPTY;
        }
    }

    return false; /* Trigger backtracking */
}

Overwriting test_sudoku.c


In [17]:
%%cmd 
gcc test_sudoku.c -o c_program


Microsoft Windows [Version 10.0.22631.4751]
(c) Microsoft Corporation. All rights reserved.

C:\Users\euhern>gcc test_sudoku.c -o c_program

C:\Users\euhern>

In [18]:
%%cmd
.\c_program.exe

Microsoft Windows [Version 10.0.22631.4751]
(c) Microsoft Corporation. All rights reserved.

C:\Users\euhern>.\c_program.exe
Initial Sudoku puzzle:
-------------------------
| 1 . . | . 7 3 | 8 9 . | 
| . . . | . . 2 | . . . | 
| . 6 . | . . . | . 1 . | 
-------------------------
| . . . | 9 . . | . . . | 
| . . 4 | . . . | . . 5 | 
| 6 . . | . 8 1 | 3 . . | 
-------------------------
| . 5 . | . 9 7 | . . 3 | 
| 7 . . | 2 . . | . . . | 
| . . . | 6 . . | 9 . . | 
-------------------------

Set E created with 945 pairs.

Solved Sudoku puzzle:
-------------------------
| 1 4 2 | 5 7 3 | 8 9 6 | 
| 8 7 9 | 1 6 2 | 5 3 4 | 
| 5 6 3 | 8 4 9 | 7 1 2 | 
-------------------------
| 3 1 7 | 9 5 4 | 2 6 8 | 
| 9 8 4 | 3 2 6 | 1 7 5 | 
| 6 2 5 | 7 8 1 | 3 4 9 | 
-------------------------
| 2 5 1 | 4 9 7 | 6 8 3 | 
| 7 9 6 | 2 3 8 | 4 5 1 | 
| 4 3 8 | 6 1 5 | 9 2 7 | 
-------------------------

C:\Users\euhern>